<a href="https://colab.research.google.com/github/SamaGhaleShah/HandWritter_Indentification/blob/main/HandWritter_Indentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
#resnet_weights_path = '/content/drive/MyDrive/CNNProject/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_dir = "/content/drive/MyDrive/CNNProject/DataTr/"
test_dir = "/content/drive/MyDrive/CNNProject/DataTe/"
# Process Train Images
train_images = []
train_labels = []
for dirs in range(1,51):
    for filename in os.listdir(train_dir + "00" + str(dirs) if dirs in range(1,10) else train_dir + "0" + str(dirs)):
        img = plt.imread(os.path.join(train_dir + "00" + str(dirs) if dirs in range(1,10) else train_dir + "0" + str(dirs),filename))
        if img is not None:
            print(str(dirs)+'/'+filename)
            train_images.append(img)
            train_labels.append("00"+str(dirs) if dirs in range(1,10) else "0"+str(dirs))
#Process Test Images
test_images = []
test_labels = []
for dirs in range(1,51):
    for filename in os.listdir(test_dir + "00" + str(dirs) if dirs in range(1,10) else test_dir + "0" + str(dirs)):
        img = plt.imread(os.path.join(test_dir + "00" + str(dirs) if dirs in range(1,10) else test_dir + "0" + str(dirs),filename))
        if img is not None:
            print(str(dirs)+'/'+filename)
            test_images.append(img)
            test_labels.append("00"+str(dirs) if dirs in range(1,10) else "0"+str(dirs))


13/Patch_16.png
13/Patch_14.png
13/Patch_17.png
13/Patch_12.png
13/Patch_121.png
13/Patch_115.png
13/Patch_120.png
13/Patch_125.png
13/Patch_123.png
13/Patch_119.png
13/Patch_124.png
13/Patch_116.png
13/Patch_110.png
13/Patch_111.png
13/Patch_126.png
13/Patch_108.png
13/Patch_113.png
13/Patch_109.png
13/Patch_100.png
13/Patch_107.png
13/Patch_112.png
13/Patch_101.png
13/Patch_104.png
13/Patch_103.png
13/Patch_122.png
13/Patch_106.png
13/Patch_102.png
13/Patch_105.png
13/Patch_114.png
13/Patch_118.png
13/Patch_117.png
13/Patch_170.png
13/Patch_156.png
13/Patch_128.png
13/Patch_168.png
13/Patch_145.png
13/Patch_152.png
13/Patch_153.png
13/Patch_171.png
13/Patch_159.png
13/Patch_143.png
13/Patch_163.png
13/Patch_167.png
13/Patch_141.png
13/Patch_166.png
13/Patch_160.png
13/Patch_130.png
13/Patch_138.png
13/Patch_146.png
13/Patch_150.png
13/Patch_140.png
13/Patch_139.png
13/Patch_137.png
13/Patch_131.png
13/Patch_134.png
13/Patch_165.png
13/Patch_144.png
13/Patch_148.png
13/Patch_127.png
1

KeyboardInterrupt: ignored

In [ ]:
"""model_resnet = ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path)
flatten1 = Flatten()(model_resnet.output)
batch1 = BatchNormalization()(flatten1)
dense1 = Dense(2048, activation='relu')(batch1)
batch2 = BatchNormalization()(dense1)
dense2 = Dense(1024, activation='relu')(batch2)
batch3 = BatchNormalization()(dense2)
dense3 = Dense(50, activation='softmax')(batch3)
model = Model(inputs=model_resnet.input, outputs=dense3)

model.layers[0].trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])"""
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras

restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(180,60,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])

#valid_data = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(180, 60),
        batch_size=30,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')
 
validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(180, 60),
        batch_size=30,
        class_mode='binary')
history = model.fit_generator(train_generator, 
                              steps_per_epoch=100, 
                              epochs=100,
                              validation_data=validation_generator, 
                              validation_steps=50, 
                              verbose=1)
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

Found 5000 images belonging to 50 classes.
Found 500 images belonging to 50 classes.
Epoch 1/50
  1/100 [..............................] - ETA: 2:48 - loss: 3.9319 - accuracy: 0.0200